<a href="https://colab.research.google.com/github/gokul-krishna-r/dm-logistic-regression/blob/main/dm_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regression is the process of relating independent variable to independent variable

Logistic regression

Dataset used here - consist of rows and columns(attributes) and th column for dependent variables

**Steps**

1. Upload dataset
2. Split the dataset- x (dependent variables), y(independent variables)
3. Learning -> Output: Model
4. Train the model
5. Evaluate the model using metrics(['accuracy'])



In [1]:
from google.colab import drive
import tensorflow as tf

drive.mount('/content/dataset/')

Mounted at /content/dataset/


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the diabetes dataset
diabetes_data = pd.read_csv('/content/dataset/MyDrive/Colab Notebooks/diabetes.csv')

# Split the dataset into features (X) and target variable (y)
X = diabetes_data.drop('Outcome', axis=1)
y = diabetes_data['Outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression(max_iter=200) #specifies the number of iterations for the algorithm to converge

# Train the model on the training data (input features -x and output - y(dependent variable))
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
# Print the evaluation results
print('Accuracy:', accuracy)

Prediction 0
Accuracy: 0.7467532467532467


In [28]:
new_data = pd.DataFrame([[8, 183, 64, 0, 0, 23.3, 0.672, 32]])
#new_data_scaled = scaler.transform(new_data)
prediction = model.predict(new_data)
if prediction == 0:
  print("Result: Not Diabetic")
else:
  print("Result: Diabetic")

Result: Diabetic


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
